In [1]:
cd "/data/ai_club/team_C/"

/data/ai_club/team_C


In [2]:
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
import joblib
from spectral import *
from PIL import Image, ImageFile
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten,Conv2D 
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras import optimizers
from tensorflow.keras.utils import plot_model
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', 

In [3]:
#Model Setup
#Data Files
data_train = np.load('CNN/spectral_lymph_tile_array_3_9_22_left_update_1.npy')
non_data_train = np.load('CNN/spectral_non_tile_array_3_9_22_left_update_1.npy')
data_test = np.load('CNN/spectral_lymph_tile_array_3_9_22_right_update_1.npy')
non_data_test = np.load('CNN/spectral_non_tile_array_3_9_22_right_update_1.npy')

data_stack_train = np.vstack((data_train, non_data_train)) # One array of all training data
data_stack_test = np.vstack((data_test, non_data_test))   # One array of all test data
labels_train = np.append(np.ones(data_train.shape[0]), np.zeros(non_data_train.shape[0])) 
labels_test = np.append(np.ones(data_test.shape[0]), np.zeros(non_data_test.shape[0]))

X_train = data_stack_train[:, :]
y_train = labels_train[:]
X_test =  data_stack_test[:, :]
y_test = labels_test[:]
print("Data Loaded")
#Model
model=Sequential()
model.add(Conv2D(filters=128,kernel_size=3,padding="same",activation="relu",input_shape=(17,17,394)))
model.add(Conv2D(filters=16,kernel_size=3,padding="same",activation="relu"))
model.add(Flatten())
model.add(Dense(500,activation="relu"))
model.add(Dense(1,activation="sigmoid"))  
print(model.summary())
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #compile Model
history=model.fit(X_train,y_train,epochs=3, validation_data=(X_test,y_test)) #train model

Data Loaded
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 17, 17, 128)       454016    
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 17, 17, 16)        18448     
_________________________________________________________________
flatten (Flatten)            (None, 4624)              0         
_________________________________________________________________
dense (Dense)                (None, 500)               2312500   
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 501       
Total params: 2,785,465
Trainable params: 2,785,465
Non-trainable params: 0
________________________________________________

In [4]:
#Loading Spectral Array and creating cropped spectral array 
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True
spectral_img = open_image('ov-63-hd-16ca.hdr')

In [24]:
def predict(size, x_start, y_start,x_num,y_num):
    for x_times in range(x_num):
        for y_times in range(y_num):
            spectral_shape=spectral_img.shape
            spectral_2d_shape=spectral_shape[:2]
            #print(spectral_2d_shape)
            mask = np.zeros((spectral_2d_shape))
            mask[(x_start+(size*(x_times))):x_start+(size*(x_times+1)),y_start+(size*(y_times)):y_start+(size*(y_times+1))]=1
            #print("Mask Shape", mask.shape)
            fig = plt.figure()
            ax = fig.add_subplot(111)
            ax.imshow(mask, aspect='equal', cmap=plt.cm.gray, interpolation='nearest') 
            #Create 17x17 tiles
            # Tile radius (1 = 3x3, 2 = 5x5, 3 = 7x7, etc.)
            tr = 8
            # Finding (y,x) tuple for where the mask is True
            y, x = np.where(mask == True)
            
            # Declaring pixel array
            tw = 1 + (tr * 2)
            test_spectral_result = np.empty((len(y), tw, tw, 394))
            
            # # Append the pixels in the spectral image to the pixel array for every (y,x) and (ny,nx) tuple
            print("Creating spectral array...")
            for index, point in enumerate(zip(y, x)):
                px, py = point[1], point[0]
                #print((index/float(len(y)))*100) # If you want to see progress for large tables
                test_spectral_result[index] = spectral_img[py-tr:py+tr+1, px-tr:px+tr+1]
            #print("Done")
            #print(test_spectral_result.shape)
            predictions=model.predict(test_spectral_result)
            #print(predictions.shape)
            predictions=predictions.reshape(size,size)
            #print(predictions.shape)
            ##fig = plt.figure()
            #ax = fig.add_subplot(111)
            #ax.imshow(predictions, aspect='auto', cmap=plt.cm.gray, interpolation='nearest')
            plt.imsave('test.png', predictions , cmap=plt.cm.gray)
            #Make a mask file out of this file for easy comparison to hyperspectral file
            output_mask = np.zeros((17912, 20292))
            output_mask[x_start:(x_start+size),y_start:(y_start+size)]=predictions
            ##print(output_mask.shape)
            fig = plt.figure(figsize = (10,10))
            #ax = fig.add_subplot(111)
            #image_file=ax.imshow(output_mask, aspect='auto', cmap=plt.cm.gray, interpolation='nearest')
            # Save the image:
            image_file = Image.fromarray(output_mask, mode="1")
            filename="predictions_mask_x_" + str(((x_start+(size*(x_times))))) +"_y_" +str(((y_start+(size*(y_times))))) +"_size_" +str(size) 
            print(filename)
            #Set colormap https://kbkb-wx-python.blogspot.com/2015/12/python-transparent-colormap.html
            colors = [(1,1,1,c) for c in np.linspace(0,1,100)]
            cmapwhite = mcolors.LinearSegmentedColormap.from_list('mycmap', colors, N=5)
            plt.imsave(f"tiled_output/{filename}_white.png", output_mask , cmap=cmapwhite)#cmap=plt.cm.gray)

In [ ]:
predict(500,13000,14000,4,4)

Creating spectral array...


In [ ]:
##Loading Spectral Array and creating cropped spectral array 
#Image.MAX_IMAGE_PIXELS = None
#ImageFile.LOAD_TRUNCATED_IMAGES = True
#spectral_img = open_image('ov-63-hd-16ca.hdr')
#print(spectral_img.shape)
#np_spectral_img=np.asarray(spectral_img)
#size=100 #Squaresize
#
#masks=np.ones((size,size,int(spectral_img.shape[0]/size),int(spectral_img.shape[1]/size)))
#print(spectral_img_cropped.shape) #(100, 100, 179, 202)
##for crop_x in range(int(spectral_img.shape[0]/size)):
##    for crop_y in range(int(spectral_img.shape[1]/size)):
##        masks[crop_x,crop_y]=np.ones(((crop_x*size):(crop_x+1)*(size),(crop_y*size):(crop_y+1)*(size)))
##        print(crop_x,crop_y)
#

In [ ]:
#ans=17912*20292
#ans2=100*100*179*202
#print(ans)
#print(ans2)
#

In [ ]:
#spectral_img_cropped=np.zeros((int(spectral_img.shape[0]/size),int(spectral_img.shape[1]/size)))
#spectral_img_cropped.shape